# SI649-20winter Lab 3 -> Altair II

## Overview 
We will continue working with the dataset from the article [“The Dollar-And-Cents Case Against Hollywood’s Exclusion of Women”](https://fivethirtyeight.com/features/the-dollar-and-cents-case-against-hollywoods-exclusion-of-women/)--don't worry, we'll switch soon. We'll focus on **transformation** and build the following charts:

1.   Line chart within a range
2.   Heatmap with additional annotations 
3.   Bar chart with additional annotation. 
4.   Bar Chart with fold 
5.   Ugliest chart

**For this lab, please write Altair code to answer the questions. In many situations you could also solve the problem using Pandas. However, we want code that can be deployed without using Python so it's better practice to just do as much as we can in Altair. You can complete the entire lab without writing any pandas transformation.**

**It's fine if your visualization looks slightly different from the example (e.g., getting 1.1 instead of 1.0, use orange instead of red)**

### Lab Instructions (read the full version on the handout of the previous lab)

*   Save, rename, and submit the ipynb file (use your username in the name).
*   Run every cell (do Runtime -> Restart and run all to make sure you have a clean working version), print to pdf, submit the pdf file. 
*   For each visualization, we will ask you to write down a "Grammar of Graphics" plan first (basically a description of what you'll code).
*   If you end up stuck, show us your work by including links (URLs) that you have searched for. You'll get partial credit for showing your work in progress. 
*   There are many bonus point opportunities in this lab. 


In [0]:
# imports we will use
import altair as alt
import pandas as pd
from altair import datum
from collections import defaultdict
datasetURL="https://raw.githubusercontent.com/LiciaHe/SI649/master/week3/movie_after_1990.csv"
movies_test=pd.read_csv(datasetURL, encoding="latin-1")


Here's the processed Bechdel test dataset. 

In [40]:
movies_test.sample(2)

,Unnamed: 0,year,title,binary,budget,dom_gross,int_gross,rating,country,language,test_result,country_binary,roi_dom,roi_int
464,464,2009,Cloudy with a Chance of Meatballs,FAIL,108573160,135575603.0,257131292.0,7.0,United States,English,Women don't talk to each other,U.S. and Canada,1.248703,1.119574
453,453,2009,Astro Boy,FAIL,70572554,21227211.0,47873178.0,6.3,Hong Kong,English,dubious,International,0.300786,0.377568


## Visualization 1: Line chart within a range

![vis1](https://github.com/LiciaHe/SI649/blob/master/week3/hw_img/1.png?raw=true)

**Description of the visualization:**

We want to see a visualization of movies that were significantly above budget relative to the mean. We'd like to know the name of the top movie that year but also to have a sense of how many other outliers (we'll defined those as at least 2x the mean budget) were produced that year.

*  plot a line chart using year and average budget for movies between 1990 and  1998. 
*  plot dots to represent movies with budgets that are at least 2 times bigger than the mean budget of that year. (e.g., if the mean budget of 1990 is 50M, plot movies made in 1990 whose budgets are at least 100M)
*  For each year, annotate the title of the movie with the highest budget of that year. 

### Visualization 1 Plan: 

TODO: edit this cell to write your visualization plan. You can write in altair syntax, in full sentence, or in bullet points, whichever way that helps you to plan your chart.  

**line chart**
* Describe the encoding rules: 
*   mark type: Bar
*   >x: year : Quantitative
*   >y: average(budget) : Nominal  
* Describe the transformations:
*   >transform_filter : datum.year<1998

**dot chart**
* Describe the encoding rules: 
*   mark type: Point
*   >x: year : Quantitative
*   >y: budget : Nominal 
* Describe the transformations:
*   >transform_joinaggregate : mean_budget= mean(budget) 
*   >transform_filter : datum.year<1998 and datum.budget>2*datum.mean_budget
*   >groupby=['year']

**text annotation**
* Describe the encoding rules: 
*   mark type: Text
*   >text="title" 
* Describe the transformations:
*   >transform_joinaggregate : max_budget= max(budget) 
*   >transform_filter : datum.budget==datum.max_budget
*   >groupby=['year']

### Replicate Vis 1 
Hint: 


*   Line chart is the simplest one, you can start from there. 
*   Think about the difference between transform_aggregate and transform_joinaggregate 


In [49]:
chart1= alt.Chart(movies_test, height=300, width=700).mark_line().encode(
    y=alt.Y("mean(budget)"),
    x=alt.X("year:Q",scale=alt.Scale(domain=(1990, 1998)),axis=alt.Axis(format='d')),
).transform_filter('datum.year<1998')

chart2= alt.Chart(movies_test, height=300, width=700).mark_circle(color="blue").encode(
    y=alt.Y("budget"),
    x=alt.X("year:Q",scale=alt.Scale(domain=(1989, 1998)),axis=alt.Axis(format='d')),
).transform_joinaggregate(mean_budget= 'mean(budget)',groupby=['year']).transform_filter('datum.year<1998').transform_filter('datum.budget>2*datum.mean_budget')

text=scatter.mark_text(align = "center").encode(
    text="title"
).transform_joinaggregate(max_budget= 'max(budget)',groupby=['year']).transform_filter('datum.budget==datum.max_budget')

chart1+chart2+text

alt.LayerChart(...)

## Visualization 2 heatmap with additional annotations


![vis2](https://github.com/LiciaHe/SI649/blob/master/week3/hw_img/2.png?raw=true)

**Description of this visualization:**

We want to produce a heatmap contrasting different types of bechdel test results to the IMDB scores of those movies. Each cell in the heatmap will tell us how many movies (not normalized here, we want raw count) are in that category. The author of the article wants to point out some property of the really well-regarded movies so wants those highlighted.

*  Plot a heatmap with rating and test result. Encode the count of movies as color. Remove the category "dubious".  
* For each cell, add text to indicate the count of movies. 
* If a test_result category has at least one movie whose rating is higher than 9, highlight that entire category. You can highlight by adding a layer of heatmap that is not filled.      

### Visualization2 Plan: 

TODO: edit this cell to write your visualization plan. You can write in altair syntax, in full sentence, or in bullet points, whichever way that helps you to plan your chart.  

**heatmap**
* mark type : rect
* Describe the encoding rules: 
*   >x: rating : Quantitative,  bin = alt.Bin(maxbins = 8)
*   >y: test_results : Nominal 
*   >color: alt.Color : count(rating):Q, scale = alt.Scale(scheme = "yellowgreenblue")

* Describe the transformations:
*   >transform_filter :  alt.FieldOneOfPredicate(field='test_result', oneOf=["Passes Bechdel Test","Women only talk about men","Women don\'t talk to each other","Fewer than two women"])   

**text annotation**
* mark type : text
* Describe the encoding rules: 
*   >text='count(movies):Q'
*   >color=alt.condition(alt.datum.rating>9)

**highlight**
* mark type : rect
* Describe the encoding rules: 
*   >x: rating : Quantitative,  bin = alt.Bin(maxbins = 8)
*   >y: test_results : Nominal   
* Describe the transformations:
*   >transform_filter :  alt.FieldOneOfPredicate(field='test_result', oneOf=["Fewer than two women"]    

### Replicate Vis 2 
Hint: 


*   When you pass the heatmap to your text chart, you are passing all of the heatmap's encoding settings as well, which include the color setting. 
*   How do you translate "at least one movie with rating > 9 " into code?   


In [15]:
chart = alt.Chart(movies_test).mark_rect().transform_filter(
     alt.FieldOneOfPredicate(field='test_result', oneOf=["Passes Bechdel Test","Women only talk about men","Women don\'t talk to each other","Fewer than two women"]) ).encode(
    alt.X('rating:Q', bin = alt.Bin(maxbins = 8)),
    alt.Y('test_result:N'),
    alt.Color(
      "count(rating):Q", 
      scale = alt.Scale(scheme = "yellowgreenblue"))
    )
chart1 = alt.Chart(movies_test).mark_rect(color='red',filled=False).transform_filter(
     alt.FieldOneOfPredicate(field='test_result', oneOf=["Fewer than two women"]) ).encode(
    alt.X('rating:Q', bin = alt.Bin(maxbins = 8)),
    alt.Y('test_result:N')
    )
text = chart.mark_text(baseline='middle').encode(
    text='count(movies):Q',
    color=alt.condition(
        alt.datum.rating>9,
        alt.value("black"),
        alt.value("black")
    )
)
chart+chart1+text

alt.LayerChart(...)

## Visualization 3: Bar chart with additional annotations

![vis3](https://github.com/LiciaHe/SI649/blob/master/week3/hw_img/3_new.png?raw=true)

**Description of this visualization:**
We want to contrast the domestic and international gross based on bedchel test passing category. We also want an example movie for each category so we're going to pick the 10th most popular one.

*  Plot a bar chart *for movies made in US and Canada*, using the test result and mean of domestic gross. 
*  For test result category (i.e., each bar), find out the movie whose domestic gross ranks as the 10th lowest (sort by ascending order, the rank = 10). Annotate the title of this movie as text. 
*  BONUS: Plot a similar bar chart *for movies made internationally* using the international gross. Concatenate the international and domestic charts charts. Make sure they share the same x axis but have independent color scales. 

### Visualization3 Plan: 

TODO: edit this cell to write your visualization plan. You can write in altair syntax, in full sentence, or in bullet points, whichever way that helps you to plan your chart.  

**bar chart**
* Describe the encoding rules: 
*   >x:  mean(dom_gross): Quantitative
*   >y:  test_result : Nominal
*   >sort=['Fewer than two women','Passes Bechdel Test','Women don\'t talk to each other','Women only talk about men']
* Describe the transformations:
*   >transform_filter : datum.country_binary == "U.S. and Canada"

**text annotation**
* Describe the encoding rules: 
*   >text : title : Nominal
* Describe the transformations::
*   >transform_window : sort=[alt.SortField('dom_gross', order='ascending')],
    rank='rank(dom_gross)',
    groupby=["test_result"]
*   >transform_filter : alt.datum.rank==10

### Replicate Vis 3 
Hint: 


*   You want to generate the rank using a transformation method that we have covered in class. 
*   It's fine if text annotations don't align perfectly with your bars. 
*   There are several transformations in this task. If your chart looks slightly different from ours, you are still likely to get full points. 



In [57]:
one=alt.Chart(movies_test).mark_bar().encode(
   y=alt.Y("test_result:N",sort=['Fewer than two women','Passes Bechdel Test','Women don\'t talk to each other','Women only talk about men']),
   x=alt.X("mean(dom_gross):Q"),
   color=alt.Color('mean(dom_gross):Q', scale=alt.Scale(scheme='blues'))
).transform_filter('datum.country_binary == "U.S. and Canada"')

# Configure rank
tc = one.mark_text(color="black", align='left',dx=3).encode(
    text=('title:N'),
)
ranking=tc.transform_window(
    sort=[alt.SortField('dom_gross', order='ascending')],
    rank='rank(dom_gross)',
    groupby=["test_result"]
).transform_filter(alt.datum.rank==10)

domt=one+ranking

two=alt.Chart(movies_test).mark_bar(color='orangered').encode(
   y=alt.Y("test_result:N",sort=['Fewer than two women','Passes Bechdel Test','Women don\'t talk to each other','Women only talk about men']),
   x=alt.X("mean(int_gross):Q"),
   color=alt.Color('mean(int_gross):Q', scale=alt.Scale(scheme='reds'))
).transform_filter('datum.country_binary == "International"')

# Configure rank
tc2 = two.mark_text(color="black", align='left',dx=3).encode(
    text=('title:N'),
)
ranking2=tc2.transform_window(
    sort=[alt.SortField('int_gross', order='ascending')],
    rank='rank(int_gross)',
    groupby=["test_result"]
).transform_filter(alt.datum.rank==10)

intr= two+ranking2

(domt &intr).resolve_scale(color='independent')


alt.VConcatChart(...)

## Visualization 4: Bar chart

![vis4](https://github.com/LiciaHe/SI649/blob/master/week3/hw_img/4.png?raw=true)

**Description of this visualization:**
* Plot a bar chart that enables the comparison between mean budget and mean international gross across the whole dataset.  
* Two bars should be colored differently   

### Visualization4 Plan: 

TODO: edit this cell to write your visualization plan. You can write in altair syntax, in full sentence, or in bullet points, whichever way that helps you to plan your chart.  


* What kind of transformation do I need to plot this bar chart? 
*   > The data has to be converted into long form with var_name='Finance', value_name='Mean of dollars' 
* What kind of encoding do I want to use?? 
*   >x:  mean(Mean of dollars): Quantitative
*   >y:  Finance : Nominal 
*   >color = alt.Color("Finance:N")



### Replicate Vis 4 
Hint: 


*   Do I need a long form or wide form data? Which type of data do I have? 




In [38]:
movies_wide=movies_test.loc[:,["Title","budget","int_gross"]].reindex()
movies_long=movies_wide.melt('Title', var_name='Finance', value_name='Mean of dollars')
c1 = alt.Chart(movies_long).mark_bar().encode(
    x=alt.X('mean(Mean of dollars):Q',title='Mean of dollars'),
    y=alt.Y('Finance:N'),
    color = alt.Color("Finance:N")
)
c1

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


alt.Chart(...)

## 5.  Ugly Visualization 

For this task, we're going to ask that you make the worst(!), ugliest, chart possible. Please use the movies dataset as a starting point, but you can add to it if you want.

You don't have to do it programatically. You can start with the Altair version or go back to Tableau, Excel, or whatever you want... You can download the chart and edit it in photoshop, illustrator, inkscape... any software you want to use. Please don't just turn in just a blank screen... :) Your chart should express **something** useful. Just do it badly.

After making the chart, please upload it to canvas (jpg or png or pdf). 

When you're done crafting your terrible chart, use the following cell to tell us what perception rules you are violating that make your chart really bad!   

### TODO: Edit this cell: Write down perception/cognition rules that your chart is violating, and why you think this chart is ugly

The plot of the task ugly vizualization is attached along with this pdf file. It's a area plot of count of different language movies by year. All different languages are color coded. From the graph only movies of english are visible. All other language movies are hardly seen. If one wants to visualize and express only about english this graph can be used, otherwise it isn't making any sence, although there is a lot of information hidden in the graph. According to perception/cognition, if the graph was made as point or circle as mark type, it would have made more sence. 

---------------------------------------------------------
*This is the end of lab 3*. 


Please run all cells (Runtime->Restart and run all), and 
1.  save to PDF (File->Print->Save PDF -> landscape, shrink to 80%)
2.  save to ipynb (File -> Download .ipynb)

Rename both files with your uniqname: e.g. uniqname.pdf/ uniqname.ipynb

Upload both files AND  the ugliest chart to canvas 


